In [1]:
import pandas as pd
import datetime as dt
import re
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.ui import Select
from webdriver_manager.chrome import ChromeDriverManager
from war_functions.pecota_tables import *
from cluster_luck_functions.cluster_luck_hitting import *
from cluster_luck_functions.cluster_luck_pitching import *
from cluster_luck_functions.cluster_luck_combined import *
from daily_adjustments.active_rosters import *
from daily_adjustments.BP_WAR import *
from daily_adjustments.todays_game_info import *
from daily_adjustments.starting_rotations_WAR import *
from daily_adjustments.adjusted_war_today import *
from odds_and_other_projections import *

In [2]:
bets = pd.read_csv('past_bets/base/bets_20220929.csv', index_col = 0)

In [3]:
bets = bets[bets.Home_Team != 'Los Angeles Angels']
bets.reset_index(drop = True, inplace = True)
bets

,Home_Team,Away_Team,Home_Prob,Away_Prob,Home_ML,Away_ML,Home_ML_Prob,Away_ML_Prob,Home_Diff,Away_Diff,Home_KC,Away_KC,Home_Bet,Away_Bet
0,Detroit Tigers,Kansas City Royals,0.624795,0.375205,-135.0,115.0,0.574468,0.465116,0.050327,-0.089912,0.011827,0.000000,1400.926439,0.000000
1,Minnesota Twins,Chicago White Sox,0.505469,0.494531,100.0,-120.0,0.500000,0.545455,0.005469,-0.050924,0.001094,0.000000,129.574705,0.000000
2,Boston Red Sox,Baltimore Orioles,0.474710,0.525290,-155.0,135.0,0.607843,0.425532,-0.133133,0.099758,0.000000,0.017365,0.000000,2056.964348
3,Chicago Cubs,Philadelphia Phillies,0.456385,0.543615,165.0,-195.0,0.377358,0.661017,0.079027,-0.117402,0.012692,0.000000,1503.412948,0.000000
4,Cleveland Guardians,Tampa Bay Rays,0.562531,0.437469,-105.0,-115.0,0.512195,0.534884,0.050336,-0.097415,0.010319,0.000000,1222.290837,0.000000
5,Milwaukee Brewers,Miami Marlins,0.744897,0.255103,-175.0,150.0,0.636364,0.400000,0.108534,-0.144897,0.029847,0.000000,3535.415414,0.000000
6,Seattle Mariners,Texas Rangers,0.434921,0.565079,-120.0,100.0,0.545455,0.500000,-0.110534,0.065079,0.000000,0.013016,0.000000,1541.761215
7,San Francisco Giants,Colorado Rockies,0.780526,0.219474,-240.0,200.0,0.705882,0.333333,0.074644,-0.113860,0.025379,0.000000,3006.192276,0.000000


In [4]:
bets.to_csv('past_bets/base/bets_20220929.csv')